### Model 1 - Enviormental sound classifiers

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import librosa
import pandas as pd
import numpy as np

import os
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Input, concatenate
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn import metrics 
# Load metadata
metadata = pd.read_csv('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/metadata/model1.csv')

In [4]:
# mfcc
max_frames = 174
def mfcc_mels(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mels = librosa.feature.melspectrogram(y=audio, sr=sample_rate)

    # Padding
    pad_width = max_frames - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    pad_width_mels = max_frames - mels.shape[1]
    mels = np.pad(mels, pad_width=((0, 0), (0, pad_width_mels)), mode='constant')

    return mfccs, mels


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import random
def display_sample_data(features):
    random_index = np.random.randint(0, len(features))
    random_mfcc = features[random_index][0]
    
    if random_mfcc.ndim > 1:
        random_mfcc = random_mfcc.squeeze()
    
    random_mfcc = np.transpose(random_mfcc)  
    
    plt.figure(figsize=(10, 4))
    sns.heatmap(random_mfcc, cmap='viridis')
    plt.title('MFCC')
    plt.show()

In [ ]:
# extract feature
for idx, row in metadata.iterrows():
    filename = os.path.join('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio',
                            'fold' + str(row['fold']), row['slice_file_name'])
    mfccs, mels = mfcc_mels(filename)
    mels_db = librosa.power_to_db(mels, ref=np.max)
    
    np.save(f'mfcc/{row["slice_file_name"]}.npy', mfccs)
    np.save(f'spectrograms/{row["slice_file_name"]}.npy', mels_db)

In [6]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, concatenate, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [7]:
def multi_input_cnn(input_shape_mfcc, input_shape_mels, num_classes):
    # mfcc
    input_mfcc = Input(shape=input_shape_mfcc)
    x = Conv2D(filters=16, kernel_size=2, activation='relu')(input_mfcc)
    x = MaxPooling2D(pool_size=2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Conv2D(filters=32, kernel_size=2, activation='relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Conv2D(filters=64, kernel_size=2, activation='relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Conv2D(filters=128, kernel_size=2, activation='relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = GlobalAveragePooling2D()(x)

    # mels
    input_mels = Input(shape=input_shape_mels)
    y = Conv2D(filters=16, kernel_size=2, activation='relu')(input_mels)
    y = MaxPooling2D(pool_size=2)(y)
    y = BatchNormalization()(y)
    y = Dropout(0.2)(y)
    y = Conv2D(filters=32, kernel_size=2, activation='relu')(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = BatchNormalization()(y)
    y = Dropout(0.2)(y)
    y = Conv2D(filters=64, kernel_size=2, activation='relu')(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = BatchNormalization()(y)
    y = Dropout(0.2)(y)
    y = Conv2D(filters=128, kernel_size=2, activation='relu')(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = BatchNormalization()(y)
    y = Dropout(0.2)(y)
    y = GlobalAveragePooling2D()(y)

    combined = concatenate([x, y])

    z = Dense(num_classes, activation='softmax')(combined)

    model = Model(inputs=[input_mfcc, input_mels], outputs=z)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [9]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

fold_accuracies = []
early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.0001, monitor='val_loss')
input_shape_mfcc = (40, 174, 1)
input_shape_mels = (128, 174, 1)
num_classes = 10
for i in range(1, 11):  # 10-fold
    print(f"Processing fold {i}")
    test_data = metadata[metadata['fold'] == i]
    train_data = metadata[metadata['fold'] != i]

    X_train_mfcc = np.array([np.load(f'mfcc/{filename}.npy') for filename in train_data['slice_file_name']])
    X_test_mfcc = np.array([np.load(f'mfcc/{filename}.npy') for filename in test_data['slice_file_name']])
    X_train_mels = np.array([np.load(f'spectrograms/{filename}.npy') for filename in train_data['slice_file_name']])
    X_test_mels = np.array([np.load(f'spectrograms/{filename}.npy') for filename in test_data['slice_file_name']])

    y_train = np.array(train_data.classID.tolist())
    y_test = np.array(test_data.classID.tolist())

    le = LabelEncoder()
    y_train = to_categorical(le.fit_transform(y_train))
    y_test = to_categorical(le.fit_transform(y_test))

    X_train_mfcc = X_train_mfcc.reshape(list(X_train_mfcc.shape) + [1])
    X_test_mfcc = X_test_mfcc.reshape(list(X_test_mfcc.shape) + [1])
    X_train_mels = X_train_mels.reshape(list(X_train_mels.shape) + [1])
    X_test_mels = X_test_mels.reshape(list(X_test_mels.shape) + [1])

    model = multi_input_cnn(input_shape_mfcc, input_shape_mels, num_classes)

    model.fit([X_train_mfcc, X_train_mels], y_train, epochs=50, batch_size=256, verbose=1)

    predictions = model.predict([X_test_mfcc, X_test_mels])
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)

    fold_accuracy = accuracy_score(true_classes, predicted_classes)
    print(f"Accuracy for fold {i}: {fold_accuracy}")
    fold_accuracies.append(fold_accuracy)

print(f"10-fold cross validation accuracy: {np.mean(fold_accuracies)}")


Processing fold 1
Epoch 1/50
28/28 [==============================] - 42s 1s/step - loss: 1.4192 - accuracy: 0.5026
Epoch 2/50
28/28 [==============================] - 45s 2s/step - loss: 0.9903 - accuracy: 0.6692
Epoch 3/50
28/28 [==============================] - 46s 2s/step - loss: 0.8244 - accuracy: 0.7274
Epoch 4/50
28/28 [==============================] - 45s 2s/step - loss: 0.7203 - accuracy: 0.7703
Epoch 5/50
28/28 [==============================] - 45s 2s/step - loss: 0.6294 - accuracy: 0.8011
Epoch 6/50
28/28 [==============================] - 48s 2s/step - loss: 0.5526 - accuracy: 0.8294
Epoch 7/50
28/28 [==============================] - 48s 2s/step - loss: 0.4994 - accuracy: 0.8425
Epoch 8/50
28/28 [==============================] - 43s 2s/step - loss: 0.4527 - accuracy: 0.8603
Epoch 9/50
28/28 [==============================] - 44s 2s/step - loss: 0.4138 - accuracy: 0.8744
Epoch 10/50
28/28 [==============================] - 43s 2s/step - loss: 0.3771 - accuracy: 0.8909
E

In [10]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 40, 174, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 input_22 (InputLayer)          [(None, 128, 174, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_80 (Conv2D)             (None, 39, 173, 16)  80          ['input_21[0][0]']               
                                                                                           